In [3]:
import json

bus_routes = json.load(open('bus_routes.json', 'r'))
bus_routes[0]

{u'@context': u'http://vocab.odpt.org/context_odpt_BusroutePattern.jsonld',
 u'@id': u'urn:ucode:_00001C000000000000010000031545B8',
 u'@type': u'odpt:BusroutePattern',
 u'dc:date': u'2017-12-07T12:33:51+09:00',
 u'dc:title': u'\u65b0\u5c0f\uff12\uff11',
 u'odpt:busLocationURL': u'https://tobus.jp/sp/blsys/stop?routecode=60&ud=1',
 u'odpt:busroute': u'odpt.Busroute:Toei.Shinko21',
 u'odpt:busstopPoleOrder': [{u'odpt:busstopPole': u'odpt.BusstopPole:Toei.Higashikomatsugawashakomae.2251.2',
   u'odpt:index': 1,
   u'odpt:note': u'\u6771\u5c0f\u677e\u5ddd\u8eca\u5eab\u524d:2251:\u6771\u5c0f\u677e\u5ddd\u8eca\u5eab\u524d2'},
  {u'odpt:busstopPole': u'odpt.BusstopPole:Toei.Funaboriekimae.2032.3',
   u'odpt:index': 2,
   u'odpt:note': u'\u8239\u5800\u99c5\u524d:2032:\u8239\u5800\u99c5\u524d3'}],
 u'odpt:direction': u'1',
 u'odpt:kana': u'\u3057\u3093\u3053\u3001\u306b\u3058\u3085\u3046\u3044\u3061',
 u'odpt:note': u'\u65b0\u5c0f\uff12\uff11:\u65b0\u5c0f\uff12\uff11\u51fa\u5165\uff0d\uff11:68

In [4]:
len(bus_routes)

887

In [5]:
# busstop_sameas = set()
# for route in bus_routes:
#     for stop in route['odpt:busstopPoleOrder']:
#         busstop_sameas.add(stop['odpt:busstopPole'])
# print len(busstop_sameas)

In [6]:
from config import API_URL, ACCESS_TOKEN
import requests
import time

def need_request(stops, bus_stops):
    for s in stops:
        if s['odpt:busstopPole'] not in bus_stops:
            return True
    return False
                
bus_stops = {}
rdf_type = 'odpt:BusstopPole'
payload = {'acl:consumerKey': ACCESS_TOKEN}
for i, route in enumerate(bus_routes):
    if not need_request(route['odpt:busstopPoleOrder'], bus_stops):
        print('pass')
        continue
    payload['odpt:busroutePattern'] =  route['owl:sameAs']
    for _ in range(5):
        try:
            r = requests.get(API_URL + rdf_type, params=payload)
            data = r.json()
        except:
            time.sleep(0.1)

    for d in data:
        bus_stops[d['owl:sameAs']] = d
    print i, len(bus_stops)

0 2
1 37
2 69
3 83
4 111
5 138
6 150
7 188
8 227
pass
pass
11 256
12 286
13 298
14 337
15 339
16 380
17 389
18 391
19 420
20 444
21 446
22 469
23 492
24 503
25 528
pass
pass
28 542
29 546
30 549
31 564
32 573
33 602
34 606
35 610
36 644
37 674
38 687
pass
40 718
41 747
42 776
43 801
44 834
45 869
46 899
47 937
48 966
49 968
50 969
pass
52 994
53 1020
54 1027
pass
56 1046
57 1058
58 1069
59 1072
60 1075
pass
62 1081
63 1148
64 1194
65 1222
pass
pass
68 1233
pass
70 1245
pass
pass
pass
74 1273
75 1281
76 1291
pass
78 1295
pass
pass
81 1299
82 1308
83 1311
pass
pass
86 1312
87 1313
88 1338
89 1351
90 1364
pass
92 1373
93 1383
pass
pass
96 1396
97 1407
98 1415
99 1428
100 1435
101 1446
102 1453
pass
pass
pass
106 1473
107 1504
108 1546
109 1555
pass
111 1568
pass
113 1575
pass
115 1582
pass
pass
118 1586
119 1594
120 1595
121 1596
pass
123 1605
pass
pass
pass
127 1621
128 1623
129 1637
130 1643
131 1649
pass
pass
pass
pass
pass
pass
pass
139 1650
pass
pass
pass
pass
144 1652
145 1659
pass


In [8]:
json.dump(bus_stops, open('bus_stops.json', 'wb'))

In [15]:
bus_stop_locs = [[stop['geo:lat'], stop['geo:long']] for stop in bus_stops.values()]

In [24]:
import folium

def plot_points(points, c='red'):
    for point in points:
        folium.CircleMarker(point, color=c, fill_color=c, radius=5, fill_opacity=0.1).add_to(mapp)

center = bus_stop_locs[0]
mapp = folium.Map(location=center, zoom_start=17)
plot_points(bus_stop_locs[:3000])
mapp

In [32]:
import pandas as pd

df = pd.DataFrame.from_dict(bus_stops, orient='index')
df.head()

,odpt:busLocationURL,odpt:busstopPoleNumber,odpt:operator,geo:long,dc:title,geo:lat,ug:region,owl:sameAs,odpt:kana,dc:date,odpt:busroutePattern,@context,odpt:note,@id,@type,odpt:busstopPoleTimetable
odpt.BusstopPole:Toei.Adachidaigochuugakkoumae.2151.1,[https://tobus.jp/sp/blsys/stop?stopid=2151&ud...,1,[odpt.Operator:Toei],139.777082,足立第五中学校前,35.777258,None,odpt.BusstopPole:Toei.Adachidaigochuugakkoumae...,あだちだいごちゅうがっこうまえ,2017-12-21T13:36:18+09:00,"[odpt.BusroutePattern:Toei.Ou40Kou.40001.1, od...",http://vocab.odpt.org/context_odpt_BusstopPole...,足立第五中学校前:2151,urn:ucode:_00001C0000000000000100000315B293,odpt:BusstopPole,[odpt.BusstopPoleTimetable:Toei.Ou40Kou.Adachi...
odpt.BusstopPole:Toei.Adachidaigochuugakkoumae.2151.2,[https://tobus.jp/sp/blsys/stop?stopid=2151&ud...,2,[odpt.Operator:Toei],139.775743,足立第五中学校前,35.776389,None,odpt.BusstopPole:Toei.Adachidaigochuugakkoumae...,あだちだいごちゅうがっこうまえ,2017-12-21T13:36:18+09:00,"[odpt.BusroutePattern:Toei.Ou40Kou.40001.2, od...",http://vocab.odpt.org/context_odpt_BusstopPole...,足立第五中学校前:2151,urn:ucode:_00001C0000000000000100000315B294,odpt:BusstopPole,[odpt.BusstopPoleTimetable:Toei.Ou40Kou.Adachi...
odpt.BusstopPole:Toei.Adachikuyakusho.2330.2,[https://tobus.jp/sp/blsys/stop?stopid=2330&ud...,2,[odpt.Operator:Toei],139.803908,足立区役所,35.775344,None,odpt.BusstopPole:Toei.Adachikuyakusho.2330.2,あだちくやくしょ,2017-12-21T13:36:18+09:00,"[odpt.BusroutePattern:Toei.Ou49.48502.1, odpt....",http://vocab.odpt.org/context_odpt_BusstopPole...,足立区役所:2330,urn:ucode:_00001C0000000000000100000315BEF7,odpt:BusstopPole,[odpt.BusstopPoleTimetable:Toei.Ou49.Adachikuy...
odpt.BusstopPole:Toei.Adachikuyakusho.2330.3,[https://tobus.jp/sp/blsys/stop?stopid=2330&ud...,3,[odpt.Operator:Toei],139.803682,足立区役所,35.775117,None,odpt.BusstopPole:Toei.Adachikuyakusho.2330.3,あだちくやくしょ,2017-12-21T13:36:18+09:00,"[odpt.BusroutePattern:Toei.Kita47.48105.2, odp...",http://vocab.odpt.org/context_odpt_BusstopPole...,足立区役所:2330,urn:ucode:_00001C0000000000000100000315BEF8,odpt:BusstopPole,[odpt.BusstopPoleTimetable:Toei.Kusa43.Adachik...
odpt.BusstopPole:Toei.Adachikuyakusho.2330.4,[https://tobus.jp/sp/blsys/stop?stopid=2330&ud...,4,[odpt.Operator:Toei],139.804094,足立区役所,35.775212,None,odpt.BusstopPole:Toei.Adachikuyakusho.2330.4,あだちくやくしょ,2017-12-21T13:36:18+09:00,"[odpt.BusroutePattern:Toei.Kita47.48102.1, odp...",http://vocab.odpt.org/context_odpt_BusstopPole...,足立区役所:2330,urn:ucode:_00001C0000000000000100000315BEF9,odpt:BusstopPole,[odpt.BusstopPoleTimetable:Toei.Kita47.Adachik...


In [34]:
df[['geo:lat', 'geo:long']].describe()

,geo:lat,geo:long
count,3701.000000,3701.000000
mean,35.702863,139.731280
std,0.049386,0.151370
min,35.579079,139.183600
25%,35.667834,139.722501
50%,35.698473,139.766949
75%,35.730761,139.809413
max,35.839711,139.903972


In [11]:
import pandas as pd
query = 'SELECT * FROM bus.sample limit 10'

df = pd.read_gbq(query, 'tokyo-odpt')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=495642085510-k0tmvj2m941jhre2nbqka17vqpjfddtd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=5uETaDxneU3ZnzTwtC2cIGklPQuXxy&prompt=consent&access_type=offline
Enter the authorization code: 4/hap_cUhum8FPfpTQUt9_gmxi7DISlKrKDTm_l9GolAs
Requesting query... ok.
Job ID: job_hx0n19YLwrMH94HODUep2qS72dO0
Query running...
Query done.
Processed: 60.5 MB
Standard price: $0.00 USD

Retrieving results...
Got 10 rows.

Total time taken 1.48 s.
Finished at 2017-12-23 16:10:13.


In [12]:
df.head()

,date,valid,busroute,busroutePattern,busNumber,operator,startingBusstopPole,terminalBusstopPole,fromBusstopPole,toBusstopPole,fromBusstopPoleTime,lat,long,azimuth,doorStatus,progress,speed
0,2017-12-17 23:50:57,2017-12-17 23:51:12,odpt.Busroute:Toei.Umi01,odpt.BusroutePattern:Toei.Umi01.72801.2,A571,odpt.Operator:Toei,odpt.BusstopPole:Toei.Toukyouterepo-toekimae.2...,odpt.BusstopPole:Toei.Monzennakachou.1544.8,odpt.BusstopPole:Toei.Daibaekimae.2329.1,odpt.BusstopPole:Toei.Fujiterebimae.2346.1,2017-12-17 23:50:27,None,None,None,None,None,None
1,2017-12-17 23:50:57,2017-12-17 23:51:12,odpt.Busroute:Toei.Umi01,odpt.BusroutePattern:Toei.Umi01.72801.2,A571,odpt.Operator:Toei,odpt.BusstopPole:Toei.Toukyouterepo-toekimae.2...,odpt.BusstopPole:Toei.Monzennakachou.1544.8,odpt.BusstopPole:Toei.Daibaekimae.2329.1,odpt.BusstopPole:Toei.Fujiterebimae.2346.1,2017-12-17 23:50:27,None,None,None,None,None,None
2,2017-12-17 23:50:57,2017-12-17 23:51:12,odpt.Busroute:Toei.Umi01,odpt.BusroutePattern:Toei.Umi01.72801.2,A571,odpt.Operator:Toei,odpt.BusstopPole:Toei.Toukyouterepo-toekimae.2...,odpt.BusstopPole:Toei.Monzennakachou.1544.8,odpt.BusstopPole:Toei.Daibaekimae.2329.1,odpt.BusstopPole:Toei.Fujiterebimae.2346.1,2017-12-17 23:50:27,None,None,None,None,None,None
3,2017-12-17 23:19:08,2017-12-17 23:19:23,odpt.Busroute:Toei.Umi01,odpt.BusroutePattern:Toei.Umi01.72801.1,A571,odpt.Operator:Toei,odpt.BusstopPole:Toei.Monzennakachou.1544.5,odpt.BusstopPole:Toei.Toukyouterepo-toekimae.2...,odpt.BusstopPole:Toei.Daibaekimae.2329.2,odpt.BusstopPole:Toei.Funenokagakukanekimae.302.2,2017-12-17 23:18:16,None,None,None,None,None,None
4,2017-12-17 23:19:08,2017-12-17 23:19:23,odpt.Busroute:Toei.Umi01,odpt.BusroutePattern:Toei.Umi01.72801.1,A571,odpt.Operator:Toei,odpt.BusstopPole:Toei.Monzennakachou.1544.5,odpt.BusstopPole:Toei.Toukyouterepo-toekimae.2...,odpt.BusstopPole:Toei.Daibaekimae.2329.2,odpt.BusstopPole:Toei.Funenokagakukanekimae.302.2,2017-12-17 23:18:16,None,None,None,None,None,None


In [7]:
date_cols = [u'dc:date', u'dct:valid', u'odpt:fromBusstopPoleTime']
schema = []
for col in cols:
    x = {}
    x["name"] = col.split(':')[-1]
    if col in date_cols:
        x["type"] = "TIMESTAMP"
    else:
        x["type"] = "STRING"
    schema.append(x)

In [8]:
import json
json.dump(schema, open("schema.json", "wb"))